In [ ]:
# Practica 4 Redes Neuronales
# Autor: Jaime del Rey, Grupo 24

In [329]:
import displayData
import pandas as p
import numpy as np
import matplotlib.pyplot as mp
import scipy.optimize as opt
from scipy.io import loadmat
from checkNNGradients import checkNNGradients

In [330]:
#Data access
data = loadmat('ex4data1.mat')
weights = loadmat('ex4weights.mat')


y = np.array(data['y']).ravel()
x = np.array(data['X'])

print(y.ravel())
print(np.shape(x))

theta1, theta2 = weights['Theta1'],weights['Theta2']


[10 10 10 ...  9  9  9]
(5000, 400)


In [331]:
#Functions
sigmoid_function = lambda a : 1 / (1 + np.vectorize(np.exp)(-a))

def backprop_cost(param_nn,ninputs,nhidden,nlabels,x,y,l):
    theta1 = np.reshape(param_nn[:nhidden*(ninputs + 1)] ,(nhidden, (ninputs + 1)))
    theta2 = np.reshape(param_nn[nhidden*(ninputs + 1):] ,(nlabels, (nhidden + 1)))
    m = len(y)
    
    x = np.hstack((np.ones((m,1)),x))
    a2 = sigmoid_function(x @ theta1.T)
    a2 = np.hstack((np.ones((m,1)),a2))
    a3 = sigmoid_function(a2 @ theta2.T)
    
    y_dummy = np.matrix(p.get_dummies(y))
    
    reg = (l/(2*m)) * (np.sum(np.sum(np.power(theta1[:,1:],2),1),0) + np.sum(np.sum(np.power(theta2[:,1:],2),1),0))
    cost = (1/m * np.sum((np.diag((-1 * y_dummy) @ np.log(a3).T) - np.diag((1 - y_dummy) @ np.log(1 - a3).T)))) + reg
    
    #Si vamos a usar la funcion de check gradient necesitamos comentar backpropagation
    return cost#, backpropagation_grad(param_nn, ninputs, nhidden , nlabels , x, y, l)

def gradient_derivative(z):
    return np.multiply(sigmoid_function(z),(1 - sigmoid_function(z)))

def random_weights(L_in, L_out):
    return np.random.uniform(-0.12,0.12,(L_out,1 + L_in))

def backpropagation_grad(param_nn, ninputs, nhidden , nlabels , x, y, reg):

    theta1 = np.reshape(param_nn[:nhidden*(ninputs + 1)] ,(nhidden, (ninputs + 1)))
    theta2 = np.reshape(param_nn[nhidden*(ninputs + 1):] ,(nlabels, (nhidden + 1)))
    m = len(y)
    gradient_1 = 0
    gradient_2 = 0
    y = np.array(p.get_dummies(y))
    
    #Si vamos a usar la funcion de check gradient, comentamos esta linea
    x = np.hstack((np.ones((m,1)),x))

    for i in np.arange(m):
        
        z2 = x[i,np.newaxis]@theta1.T
        z2 = np.ravel(z2)
        z3 = np.hstack((np.ones(1), sigmoid_function(z2)))@theta2.T
        a3 = sigmoid_function(z3)  
        
        d3 = a3 - y[i,:]

        z2 = np.hstack((np.ones(1),z2))
        d2 = (d3 @ theta2) * gradient_derivative(z2)
        
        
        gradient_1 = gradient_1 + (np.matrix(d2[1:]).T @ x[i,np.newaxis])
        gradient_2 = gradient_2 + d3[np.newaxis,:].T @ np.hstack((np.ones(1), sigmoid_function(z2[1:])))[np.newaxis,:]
    
    gradient_1 /= m
    gradient_2 /= m
    
    gradient_1[:,1:] += theta1[:,1:] * (reg/m) 
    gradient_2[:,1:] += theta2[:,1:] * (reg/m) 

    return np.hstack((np.ravel(gradient_1), np.ravel(gradient_2)))

In [305]:
#Check cost function
nn_params = np.hstack((theta1.ravel(),theta2.ravel()))
cost = backprop_cost(nn_params,400,25,10,x,y,1)
print(cost)

(0.38376985909092365,)


In [315]:
#Check gradient
checkNNGradients(backprop_cost, 3)


array([ 5.05556846e-11,  2.85782509e-12,  6.26221297e-12,  1.31206157e-11,
       -6.30465263e-11, -6.03961325e-14, -9.00912678e-12, -5.12656584e-11,
       -9.29989974e-11,  7.12857551e-12, -3.97737399e-11, -1.21838206e-10,
       -1.95650579e-11,  8.77076189e-13, -8.97304453e-12, -2.29569697e-11,
        2.37940986e-11,  1.67521552e-12,  9.77740458e-12,  2.16938689e-11,
        6.03760375e-11,  1.50165991e-11,  1.18947074e-11,  5.34317035e-12,
        2.29757879e-11,  2.02671213e-11,  6.93309854e-11,  1.34677824e-11,
        2.70894418e-12,  1.42962309e-11,  1.96933581e-11,  2.04861683e-11,
        7.99529332e-11,  1.80326865e-11,  1.28570210e-11,  2.22010188e-11,
        1.16048837e-11,  1.68222103e-11])

In [332]:
#Check implementation
theta1 = random_weights(400,25)
theta2 = random_weights(25,10)
nn_params = np.hstack((theta1.ravel(),theta2.ravel()))
res = opt.minimize(backprop_cost,x0=nn_params,args=(400,25,10,x,y,1),method='TNC',options={'maxiter': 70},jac=backpropagation_grad)

In [327]:
#Accuracy
theta1 = np.reshape(res['x'][:25*(400 + 1)] ,(25, (400 + 1)))
theta2 = np.reshape(res['x'][25*(400 + 1):] ,(10, (25 + 1)))
m = len(y)
x = np.hstack((np.ones((np.shape(x)[0],1)),x))
a2 = sigmoid_function(x @ theta1.T)
a2 = np.hstack((np.ones((m,1)),a2))
a3 = sigmoid_function(a2 @ theta2.T)
np.sum(np.ravel(np.argmax(a3, axis = 1) + 1) == data['y'].T)/m

0.9166